In [3]:
def PlotAndCompare(x, y, ths = [], xlbl = "x", ylbl = "y", title_i = ""):
    '''
        Plots experimental value versus possible theoretical predictions.
        Parameters:
            - x = absciss of the values
            - y = experimental values
            - ths = list of function of 'x' for theoretical predictions
            - xlbl = label of the x-axis in the plot
            - ylbl = label of the y-axis in the plot
            - title_i = title of the plot
    '''
    ax=matplotlib.pyplot.gca()

    plt.plot(x,y,'b.',lw=2, label = "Numerical")
    i = 0
    x = x[::10]
    clrs = ['red', 'orange', 'green']
    for th in ths:
        i += 1
        plt.plot(x, th(x), '.', color = clrs[i%3],label = "Theoretical " + str(i))

    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(1.5)
    ax.tick_params('both', length=8, width=1.5, which='major')
    ax.tick_params('both', length=4, width=1.5, which='minor')
    ax.tick_params(direction='in')
    xlabel(xlbl,fontsize=30)
    ylabel(ylbl,fontsize=30)
    title(title_i, fontsize=30)
    xticks(size=25)
    yticks(size=25)
    legend()
    show()

In [4]:
def detect_crossing(Ltab, IPR):
    '''
        Detects the index at which the IPR*sqrt(L) crosses for the different values of L,
        taking for reference the biggest L.
        Parameters:
            - Ltab: List of lengths for which the IPR was computed
            - IPR: Dictionnary that assigns to each length a list of IPR values
    '''
    refL = Ltab[-1]
    return [np.argwhere(np.diff(np.sign(IPR[l]*np.sqrt(l) - IPR[refL]*np.sqrt(refL)))).flatten()[0]
            for l in Ltab[:-1]]

In [5]:
def compute_Vc(Vtab, crossing):
    '''
        Computes the critical potential from the crossing index given by detect_crossing
        Parameters:
            - Vtab: List of potentials for which the IPR was computed
            - crossing: Result of detect_crossing, i.e. index of IPR for which the crossing happens.
    '''
    minC = min(crossing)
    maxC = max(crossing)
    step = Vtab[1] - Vtab[0]
    error = max(2*step, (Vtab[maxC] - Vtab[minC])/2)
    Vc = (Vtab[maxC] + Vtab[minC] + step/2)/2
    display(Latex('$V_c = {0} \pm {1}$'.format(Vc, error)))
    return Vc, error

In [6]:
def plot_alpha(Vtab, Ltab, IPR, ymaxTab= [4, 4, 10, 50, 100]):
    '''
        Plot the IPR*L^alpha for different values of alpha and different values of L. The values of alpha
        taken are: 1/4, 1/3, 1/2, 2/3, 3/4.
        Parameters:
            - Vtab: List of potentials for which the IPR was computed
            - Ltab: List of lengths for which the IPR was computed
            - IPR: Dictionarry that maps each length to a list of IPR where the i-th element of the list
                   corresponds to the i-th potential of Vtab.
            - ymaxTab: list of the maximum values of the y-axis for each value of alpha
    '''
    alphaTab = [1/4, 1/3, 1/2, 2/3, 3/4]
    i = 0
    fig, axes = plt.subplots(3, 2,figsize=(15,15))
    for alpha in alphaTab:
        for l in Ltab:
            axes[i%3, i//3].plot(Vtab, IPR[l]*l**alpha, label='L = {}'.format(l))
        if ymaxTab != None: axes[i%3, i//3].set_ylim(0, ymaxTab[i])
        axes[i%3, i//3].legend()
        axes[i%3, i//3].set_title(r'Plot for $\alpha={}$'.format(alpha))
        axes[i%3, i//3].set_ylabel(r'$IPR_0 L^{\alpha}a^{1-\alpha}$')
        axes[i%3, i//3].set_xlabel(r'$V/E_r$')
        i += 1
    fig.show()

In [7]:
def fit_nu(Vtab, Vc, IPR, Ltab):
    '''
        Finds the exponential parameter close but higher than V_c.
        Parameters:
            - Vtab: List of computed potentials
            - Vc: Computed value of V_c for the problem.
            - IPR: Dictionarry that maps each length to a list of IPR where the i-th element of the list
                   corresponds to the i-th potential of Vtab.
            - Ltab: List of lengths for which the IPR was computed
    '''
    def func(V, a, b):
        return a*V + b
    popt, pcov = curve_fit(func, np.log(Vtab-Vc), np.log(IPR[Ltab[-1]]))
    perr = np.sqrt(np.diag(pcov))
    return popt[0], popt[1], perr[0], perr[1]

In [8]:
def plot_nu(Vc, Ltab, f, minS = 1e-2, maxS = 1e-2, Delta = 1e-2):
    '''
        Recomputes Vc right next to the previously computed value then plots the linear fit given by 
        fit_nu in log-log plot close to the transition.
        Parameters:
            - Vc: Pre-computed value of V_c
            - Ltab: List of lengths for which the IPR was computed
            - f: Function that takes for input a potential and a length and outputs a potclass
            - minS: left-delta-change for which V_c will be recomputed
            - maxS: right-delta-change for which V_c will be recomputed
            - Delta: shift right from V_c from which to start plotting and fitting
    '''
    IPR_c = {}
    count = 0
    Vtab_c = np.linspace(Vc-minS,Vc+maxS,100)
    print(Vc-minS, Vc+maxS)
    res = (maxS + minS)/100
    for l in Ltab:
        IPR_c[l] = []
        count = 0
        for V in Vtab_c:
            if count%10 == 0:
                print(count, end = "%, ")
            Lattice2DInc = f(V, l)
            IPR_c[l].append(Lattice2DInc.IPR[0])
            count += 1
        print('100%')
    
    for l in Ltab:
        IPR_c[l] = np.array(IPR_c[l])
    
    crossing = detect_crossing(Ltab, IPR_c)
    print(crossing)
    Vc, err = compute_Vc(Vtab_c, crossing)
    
    idx1 = max(crossing) + int(Delta/res)
    print(idx1)
    
    Vtab_c = Vtab_c[idx1:]
    for l in Ltab:
        IPR_c[l] = IPR_c[l][idx1:]
    
    a,b,err_a,err_b = fit_nu(Vtab_c, Vc, IPR_c, Ltab)

    for l in Ltab:
        plt.loglog(Vtab_c-Vc, IPR_c[l], label = r'$L = {}$'.format(l))
    plt.loglog(Vtab_c-Vc, np.exp(a*np.log(Vtab_c-Vc) + b), label = r'$y = {0}x + {1}$'.format(round(a,3), round(b,3)))
    plt.xlabel(r'$(V-V_c)/E_r$')
    plt.ylabel(r'$IPR_0\times a$')
    plt.title(r'Log-log plot of the $IPR_0$ close to the phase transition.')
    plt.legend()
    plt.show()
    display(Latex('$\nu = {0} \pm {1}$'.format(a, err_a)))
    return a,b,err_a,err_b

In [9]:
def plot_nuprime(Ltab, f, Vc, Vmin, Vmax):
    '''
    Same concept to plot_nu() but made simpler for fits far from transition.
    Parameters:
        - Ltab: List of lengths for which the IPR was computed
        - f: Function that takes for input a potential and a length and outputs a potclass
        - Vc: Precomputed value of Vc
        - Vmin: Left bound for the potential where the IPR will be computed and the fit will be performed.
        - Vmax: Right bound for the potential where the IPR will be computed and the fit will be performed.
    '''
    IPR_c = {}
    count = 0
    Vtab_c = np.linspace(Vmin, Vmax,100)
    for l in Ltab:
        IPR_c[l] = []
        count = 0
        for V in Vtab_c:
            if count%10 == 0:
                print(count, end = "%, ")
            Lattice2DInc = f(V, l)
            IPR_c[l].append(Lattice2DInc.IPR[0])
            count += 1
        print('100%')
    
    for l in Ltab:
        IPR_c[l] = np.array(IPR_c[l])
    
    a,b,err_a,err_b = fit_nu(Vtab_c, Vc, IPR_c, Ltab)

    for l in Ltab:
        plt.loglog(Vtab_c, IPR_c[l], label = r'$L = {}$'.format(l))
    plt.loglog(Vtab_c-Vc, np.exp(a*np.log(Vtab_c-Vc) + b), label = r'$y = {0}x + {1}$'.format(round(a,3), round(b,3)))
    plt.xlabel(r'$(V-V_c)/E_r$')
    plt.ylabel(r'$IPR_0\times a$')
    plt.title(r'Log-log plot of the $IPR_0$ far from the phase transition.')
    plt.legend()
    plt.show()
    display(Latex("$\nu' = {0} \pm {1}$".format(a, err_a)))
    return a,b,err_a,err_b

In [12]:
def Bin(epsilon, func, start, threshold = 0.05):
    '''
    Returns binary 1 if maximum value in func in between index start and index start_epsilon is bigger than
    the given threshold.
    Parameters:
        - epsilon: Number of indexes contained in one bin.
        - func: Array on which the study is performed.
        - start: Starting index of the bin.
        - threshold (default 0.05): Critical value above which we consider existence.
    '''
    if max(func[start:start+epsilon]) > threshold:
        return 1
    else:
        return 0

In [13]:
def N(func, epsTab, threshold = 0.05):
    '''
    Returns a list of values. The i-th list corresponds to the i-th value of epsilon in epsTab and corresponds
    to the number of bins that turned out to be full (+1 to avoid a log(0) error).
    Parameters:
        - func: Array that is being studied.
        - epsTab: List of epsilon for which the computation will be done.
        - threshold (default 0.05): Critical value above which we consider existence.
    '''
    out = []
    for epsilon in epsTab:
        tmp = []
        for start in range(int(len(func)/epsilon)-1):
            tmp.append(Bin(epsilon, func, start*epsilon, threshold))
        tmp = np.array(tmp)
        out.append(sum(tmp)+1)
    return out

In [14]:
def BoxDim(func, epsTab, threshold = 0.05, plot = False):
    '''
    Returns the approximate Box Dimension of the func array.
    Parameters:
        - func: Array to be studied.
        - epsTab: List of epsilons to be used for the computation.
        - threshold (default 0.05): Critical value above which we consider existence.
        - plot (default False): if set to True then this function will also plot N(epsilon) for 
                                the computed values.
    '''
    Neps = N(func, epsTab, threshold)
    Neps = np.array(Neps)
    if plot:
        plt.loglog(epsTab, Neps)
        plt.show()
    def fit(eps, a, b):
        return a*eps + b
    popt, pcov = curve_fit(fit, np.log(epsTab), np.log(Neps))
    perr = np.sqrt(np.diag(pcov))
    return popt[0], perr[0]

In [ ]:
def IPRq_L_V(Ltab, Vtab, q = 2):
    '''
    Function that computes a dictionnary that maps a given potential to a list of IPR_q where the i-th 
    item of the list corresponds to the IPR_q of the i-th length in Ltab.
    Parameters:
        - Ltab: List of lengths for which to compute the IPR.
        - Vtab: List of potentials for which to compute the IPR.
        - q (default 2): Value of q for the IPR_q
    '''
    if Ltab is None:
        LtabLog = np.linspace(np.log10(50), np.log10(10000), 1000)
        Ltab = np.power(10, LtabLog)
    if Vtab is None:
        Vtab = np.linspace(1.1, 1.12, 100)
    IPR = {}
    count = 0

    for V in Vtab:
        if count%1 == 0:
            print('{}%, '.format(count), end='')
        IPR[V] = []
        for L in Ltab:
            Lattice = pot.wVsin(4,(np.sqrt(5)-1)/2,1/100,L,V,V,"L","lnrhighbound", "nm_2", "q_{}".format(q))
            IPR[V].append(Lattice.IPR[0])
        count += 1
    return IPR

In [ ]:
def tauq(Ltab, Vtab, IPRq, p1 = 0, p2 = 1):
    '''
    Function that computes tau_q from the dictionnary of IPR_q, given a certain cut of Ltab.
    Parameters:
        - Ltab: Values of L for which the IPR was computed
        - Vtab: Values of V for which the IPR was computed
        - IPRq: Dictionnary that maps a given potential to a list of IPR where the i-th item of the
                list corresponds to the IPR of the i-th length in Ltab.
        - p1: Lower bound of Ltab defining the region in which we perform the fit to compute tau (
              default is 0%)
        - p2: Upper bound of Ltab defining the region in which we perform the fit to compute tau (
              default is 100%)
    '''
    def func(L, a, b):
        return a*L + b
    cnt = len(Ltab)
    idx1 = int(np.round(p1*cnt))
    idx2 = int(np.round(p2*cnt))-1
    tau = []
    err_tau = []
    for V in Vtab:
        redL = Ltab[idx1:idx2]
        redIPR = IPRq[V][idx1:idx2]
        popt, pcov = curve_fit(func, np.log(redL), np.log(redIPR))
        tau.append(-popt[0])
        err_tau.append(np.sqrt(np.diag(pcov))[0])
    return tau, err_tau

In [ ]:
def cuts_tauq(Ltab, Vtab, IPRq, cuts = [(0, 1), (0.2, 1), (0.4, 1), (0.6, 1), (0.8, 1)]):
    '''
    Function that computes tau by fitting the IPRq of selected portions of Ltab.
    Parameters:
        - Ltab: List of lengths for which the IPRq was computed.
        - Vtab: List of potentials for which the IPRq was computed.
        - IPRq: Dictionnary that maps a given potential to a list of IPR where the i-th item of the
                list corresponds to the IPR of the i-th length in Ltab.
        - cuts (default [(0, 1), (0.2, 1), (0.4, 1), (0.6, 1), (0.8, 1)]): List of different cuts of Ltab on
                which to perform the fit. Each value is a tuple (p1, p2) where p1 correspond to the left bound
                of Ltab (in percentage) and p2 the right bound of Ltab (in percentage) on which to perform the
                fit.
                
    '''
    tauTab = []
    err_tauTab = []
    for (p1, p2) in cuts:
        tau, err_tau = tauq(Ltab, Vtab, IPRq, p1, p2)
        tauTab.append(tau)
        err_tauTab.append(err_tau)
    return tauTab, err_tauTab

In [ ]:
def plot_tauq(Ltab, Vtab, IPRq, cuts = [(0, 1), (0.2, 1), (0.4, 1), (0.6, 1), (0.8, 1)]):
    '''
    Compute and plot tau with error bars as a function of the potential. 
    Parameters:
        - Ltab: List of lengths for which the IPRq was computed.
        - Vtab: List of potentials for which the IPRq was computed.
        - IPRq: Dictionnary that maps a given potential to a list of IPR where the i-th item of the
                list corresponds to the IPR of the i-th length in Ltab.
        - cuts (default [(0, 1), (0.2, 1), (0.4, 1), (0.6, 1), (0.8, 1)]): List of different cuts of Ltab on
                which to perform the fit. Each value is a tuple (p1, p2) where p1 correspond to the left bound
                of Ltab (in percentage) and p2 the right bound of Ltab (in percentage) on which to perform the
                fit.
    '''
    fig = plt.figure(figsize=(15,10))
    tauTab = []
    err_tauTab = []
    for (p1, p2) in cuts:
        tau, err_tau = tauq(Ltab, Vtab, IPRq, p1, p2)
        tauTab.append(tau)
        err_tauTab.append(err_tau)
        plt.errorbar(Vtab, tau, yerr = err_tau, label = 'Cut: ({0}, {1})'.format(p1, p2))
    plt.xlabel(r'$V/E_r$', size = 20)
    plt.ylabel(r'$\tau_q$', size = 30)
    plt.legend()
    plt.title(r'Evolution of $\tau_q$ as a function of $V/E_r$.', size = 20)
    plt.show()
    return tauTab, err_tauTab

In [ ]:
def plot_ground_wavefunc(V, L, exp = 2):
    '''
    Plots the wavefunction of the groundstate for given input values.
    Parameters:
        - V: Potential for which to compute the problem.
        - L: Length to use for the computation.
        - exp (default 2): Power to which the wavefunction will be put.
    '''
    Lattice = pot.wVsin(4,(np.sqrt(5)-1)/2,1/100,L,V,V,"L","lnrhighbound", "nm_2", "q_3")
    absi=np.ones(Lattice.n)
    for i in range(Lattice.n):
        absi[i]=Lattice.dx*i

    fig = plt.figure(figsize=(15,10))
    plt.xlabel(r'$x$', size = 20)
    plt.ylabel(r'$|\psi(x)|^{}$'.format(exp), size = 20)
    plt.plot(absi,np.power(Lattice.Ve[:,0],exp)/((np.sin(np.pi*absi/L)**2)))
    plt.plot()
    plt.show()
    
    return absi, np.power(Lattice.Ve[:,0], exp)/((np.sin(np.pi*absi/L)**2))